In [ ]:
#%pip install gym[atari]
#%pip install gym[all]
#%pip install opencv-python
#%pip install gym[accept-rom-license]
#%pip install gym pyvirtualdisplay matplotlib

In [ ]:
import gym
from gym import wrappers
import matplotlib.pyplot as plt
from IPython import display
import random
import cv2
import pyNN.spiNNaker as p
p.setup(0.1)
import math

In [ ]:
#########
# Input #
#########

In [ ]:
#
#
#
#

In [ ]:
def inputFrameProcessor(frame):
    return cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)


In [ ]:
def inputToSpikeSourceArray(frame):
    grayframe = inputFrameProcessor(frame) 
    return [m+100 for line in grayframe for m in line]

In [ ]:
def createOutputLayers(action_space_size):
    output=[]
    for i in range(action_space_size):
        l = p.Population(10,p.IF_curr_exp())
        output.append(l)
        l.record(["spikes", "v"])
    return output

In [ ]:
def selectOutput(records):
    choice = 0
    best = 0
    for i in range(len(records)):
        if len(records[i])>best:
            best = len(records[i])
            choice = i
    return choice
        
    

In [ ]:
timing_rule = p.SpikePairRule(tau_plus=0.1, tau_minus=0.1, A_plus=0.1, A_minus=0.1)
weight_rule_pos = p.AdditiveWeightDependence(w_max=50.0, w_min=1.0)
weight_rule_neg = p.AdditiveWeightDependence(w_max=1.0, w_min=-50.0)
stdp_model_excitatory = p.STDPMechanism(timing_dependence=timing_rule, weight_dependence=weight_rule_pos, weight=25.0,delay=5.0)
stdp_model_inhibitory = p.STDPMechanism(timing_dependence=timing_rule, weight_dependence=weight_rule_neg, weight=-25.0,delay=5.0)

In [ ]:
env = gym.make("Pong-v0")
print("The environment has the following {} actions: {}".format(env.action_space.n, 
                                                                env.unwrapped.get_action_meanings()))
env = wrappers.Monitor(env, "/tmp/Pong-v0", force=True)
print(len(env.unwrapped.get_action_meanings()))
fig = plt.figure(figsize=(5, 5))
img = plt.imshow(env.render(mode='rgb_array'))

In [ ]:


### INPUT LAYER ###
inLayer = p.Population(len(inputToSpikeSourceArray(env.render(mode='rgb_array'))),p.SpikeSourcePoisson(rate=inputToSpikeSourceArray(env.render(mode='rgb_array'))))
inLayer.record(["spikes"])
### EXCITATION LAYER ###
excitatoryPop = p.Population(int(10),p.IF_curr_exp())
excitatoryPop.record(["spikes"])
### INHIBITORY LAYER ###
inhibitoryPop = p.Population(int(10),p.IF_curr_exp())
inhibitoryPop.record(["spikes"])
### REWARD POPULATION ###
rewardInput = p.Population(10,p.SpikeSourcePoisson(rate=0))
### PUNISHMENT POPULATION ###
punInput = p.Population(10,p.SpikeSourcePoisson(rate=0))
### Output Layers ###
outputLayers = createOutputLayers(len(env.unwrapped.get_action_meanings()))


### PROJECTION ###
InputExcProjection = p.Projection(inLayer,excitatoryPop,p.AllToAllConnector(), synapse_type=stdp_model_excitatory)
#excInhProjection = p.Projection(excitatoryPop,inhibitoryPop,p.AllToAllConnector(), synapse_type=stdp_model_excitatory)
#inhExcProjection = p.Projection(inhibitoryPop,excitatoryPop,p.AllToAllConnector(), synapse_type=stdp_model_excitatory)
rewardExcProjection = p.Projection(rewardInput,excitatoryPop,p.AllToAllConnector())
#punInhProjection = p.Projection(punInput,inhibitoryPop,p.AllToAllConnector())
punExcProjection = p.Projection(punInput,excitatoryPop,p.AllToAllConnector())
outExcProjections = [p.Projection(excitatoryPop,outputLayer,p.AllToAllConnector()) for outputLayer in outputLayers]



#action = random.randint(0,5)
episodes=[*range(2)]
totreward = []
for episode in range(2):
    observation = env.reset()
    
    step = 0
    total_reward = 0
    action=0
    done =False
   
    while not done:
        step += 1
        
        env.render()
        print("episode",episode,"step",step,"reward",total_reward,"action",action)
        
        img.set_data(env.render(mode='rgb_array'))
        
        display.display(plt.gcf())
        display.clear_output(wait=True)
        
        
        observation, reward, done, info = env.step(action)
        inLayer.set(rate=inputToSpikeSourceArray(observation))
        rewardInput.set(rate= 1 if total_reward<0 else total_reward)
        punInput.set(rate = abs(total_reward) if total_reward<0 else 1) 
        print([outputLayer.get_data("spikes").segments[0].spiketrains[0] for outputLayer in outputLayers])
        print("inputspikes",inLayer.get_data("spikes").segments[0].spiketrains[0])
        print("exspikes",excitatoryPop.get_data("spikes").segments[0].spiketrains[0])
        #print("inspikes",inhibitoryPop.get_data("spikes").segments[0].spiketrains[0])
        #print("WEGHITS",InputExcProjection.get(["weight"], "list"))
        p.run(100)
        
        action = selectOutput([outputLayer.get_data("spikes").segments[0].spiketrains[0] for outputLayer in outputLayers])
        
        
        #print("WEGHITS",InputExcProjection.get(["weight"], "list"))
       
        total_reward += reward
        
   
        
        
    totreward.append(total_reward)
    print("Episode: {0},\tSteps: {1},\tscore: {2}".format(episode, step, total_reward))
    
env.close()

In [ ]:
plt.plot(episodes,totreward)
plt.xlabel('Episodes')
plt.ylabel('Total Reward')
plt.savefig("TrainingResult.png")

In [ ]:
##################
# Neural Network #
##################
#
#
#
#

In [ ]:
####### 

In [ ]:
##########
# Output #
##########
#
#
#
#